In [48]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random

In [18]:
data = pd.read_csv('/Users/yonghui/Desktop/FashionDataset.csv')
data.replace(['NaN', 'None'], np.nan, inplace=True)
df = data.dropna()
data = data.sample(10000)
data

,ProductID,BrandName,Deatils,Sizes,MRP,SellPrice,Discount,Category
4333,4334,zink london,textured polyester v neck womens top - pink,"Size:Large,Medium,Small,X-Large,X-Small",Rs\n1599,959,40% off,Westernwear-Women
2304,2305,only,solid cotton womens t-shirt - yellow,"Size:Large,Medium,Small,X-Large,X-Small",Rs\n1799,1079,40% off,Westernwear-Women
16600,12,kami kubi,embroidered silk blend womens unstitched suit ...,Nan,Rs\n3999,1599,60% off,Indianwear-Women
5245,5246,forever new,solid polyester halter neck womens top - blush,"Size:Large,Medium,Small,X-Small,XX-Small",Nan,3400,Nan,Westernwear-Women
24880,24,inc.5,suede slip on womens casual slides - pistachio,"Size:4,5,6,7,8,9,10",Rs\n2290,1190,48% off,Footwear-Women
...,...,...,...,...,...,...,...,...
30610,8,giva,sterling silver cubic zirconia womens western ...,Nan,Rs\n499,474,5% off,Jewellery-Women
30386,18,ayesha,womens drop earrings with pastel stone - multi,Nan,Rs\n698,349,50% off,Jewellery-Women
19849,11,indya,floral silk regular fit womens sharara - pink,"Size:Large,Small,X-Large",Nan,2450,Nan,Indianwear-Women
24910,2,inc.5,synthetic slipon womens party wear flats - pal...,"Size:37,38,39,40",Rs\n1890,945,50% off,Footwear-Women


In [19]:
data.columns

Index(['ProductID', 'BrandName', 'Deatils', 'Sizes', 'MRP', 'SellPrice',
       'Discount', 'Category'],
      dtype='object')

In [20]:
data.isnull().sum()

ProductID    0
BrandName    0
Deatils      0
Sizes        0
MRP          0
SellPrice    0
Discount     0
Category     0
dtype: int64

In [29]:
data["ProductID"] = data.index + 1
data = data.rename(columns = {'Deatils' : 'Details'})

In [42]:
new_data = data[['ProductID','BrandName','Details','Category']]
new_data

,ProductID,BrandName,Details,Category
4333,4334,zink london,textured polyester v neck womens top - pink,Westernwear-Women
2304,2305,only,solid cotton womens t-shirt - yellow,Westernwear-Women
16600,16601,kami kubi,embroidered silk blend womens unstitched suit ...,Indianwear-Women
5245,5246,forever new,solid polyester halter neck womens top - blush,Westernwear-Women
24880,24881,inc.5,suede slip on womens casual slides - pistachio,Footwear-Women
...,...,...,...,...
30610,30611,giva,sterling silver cubic zirconia womens western ...,Jewellery-Women
30386,30387,ayesha,womens drop earrings with pastel stone - multi,Jewellery-Women
19849,19850,indya,floral silk regular fit womens sharara - pink,Indianwear-Women
24910,24911,inc.5,synthetic slipon womens party wear flats - pal...,Footwear-Women


In [43]:
new_data['tags'] = new_data['BrandName'] + " " + new_data['Details'] + " " + new_data['Category']
new_data = new_data[['ProductID', 'tags']]
new_data

<ipython-input-43-e15a82069d33>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['tags'] = new_data['BrandName'] + " " + new_data['Details'] + " " + new_data['Category']


,ProductID,tags
4333,4334,zink london textured polyester v neck womens t...
2304,2305,only solid cotton womens t-shirt - yellow West...
16600,16601,kami kubi embroidered silk blend womens unstit...
5245,5246,forever new solid polyester halter neck womens...
24880,24881,inc.5 suede slip on womens casual slides - pis...
...,...,...
30610,30611,giva sterling silver cubic zirconia womens wes...
30386,30387,ayesha womens drop earrings with pastel stone ...
19849,19850,indya floral silk regular fit womens sharara -...
24910,24911,inc.5 synthetic slipon womens party wear flats...


In [56]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

def preprocess_data(tiktok_data):
    records = []
    for idx, row in tiktok_data.iterrows():
        for user_id in row['user_id']:
            records.append((row['video_id'], user_id, row['product_tags']))
    return pd.DataFrame(records, columns=['video_id', 'user_id', 'product_tags'])

def extract_product_features(tiktok_data):
    #create a TFIDF vectorizer for the product tags
    vectorizer = TfidfVectorizer()
    product_features = vectorizer.fit_transform(tiktok_data['product_tags'])
    return product_features


def train_recommendation_model(tiktok_data):

    reader = Reader(rating_scale=(0, 1))
    tiktok_data['rating'] = 1
    data = Dataset.load_from_df(tiktok_data[['user_id', 'video_id', 'rating']], reader)
    trainset, testset = train_test_split(data, test_size=0.2)
    algo = SVD()
    algo.fit(trainset)
    
    predictions = algo.test(testset)
    accuracy.rmse(predictions)
    
    return algo


def recommend_products(user_id, model, tiktok_data, product_features):
    user_data = tiktok_data[tiktok_data['user_id'] == user_id]
    liked_product_features = product_features[user_data.index]
    similarity = cosine_similarity(liked_product_features, product_features)
    similarity_scores = similarity.mean(axis=0)
    
    recommended_indices = similarity_scores.argsort()[-5:][::-1]
    
    recommended_products = tiktok_data.iloc[recommended_indices]['product_tags'].values
    return recommended_products

if __name__ == "__main__":
    tiktok_data = new_data
    
    tiktok_data = preprocess_data(tiktok_data)
    product_features = extract_product_features(tiktok_data)
    model = train_recommendation_model(tiktok_data)
    
    user_id = 1
    recommendations = recommend_products(user_id, model, tiktok_data, product_features)
    print("Recommended Products:", recommendations)


RMSE: 0.0009
Recommended Products: ['w solid cotton round neck womens kurta - pink Indianwear-Women'
 'w printed cotton round neck womens straight fit kurta - blue Indianwear-Women'
 'w printed cotton round neck womens straight fit kurta - blue Indianwear-Women'
 'w printed cotton round neck womens straight fit kurta - blue Indianwear-Women'
 'w printed cotton round neck womens straight fit kurta - blue Indianwear-Women']


In [53]:
tiktok_data

,video_id,user_id,product_tags,rating
0,1,1,"laptop,phone",1
1,1,2,"laptop,phone",1
2,2,1,"phone,headphones",1
3,4,2,"laptop,tablet",1
4,4,3,"laptop,tablet",1
5,5,3,"phone,headphones",1


In [55]:
new_data = new_data.rename(columns = {'ProductID': 'video_id'})
new_data['user_id'] = random_users
new_data = new_data.rename(columns = {'tags': 'product_tags'})
new_data

,video_id,product_tags,user_id
4333,4334,zink london textured polyester v neck womens t...,"[3, 7, 2]"
2304,2305,only solid cotton womens t-shirt - yellow West...,"[10, 3, 10, 10, 3, 9, 8, 9]"
16600,16601,kami kubi embroidered silk blend womens unstit...,"[4, 9, 5, 3, 4, 2, 3, 7, 1]"
5245,5246,forever new solid polyester halter neck womens...,"[8, 6, 2, 8, 7]"
24880,24881,inc.5 suede slip on womens casual slides - pis...,"[8, 9, 3, 2]"
...,...,...,...
30610,30611,giva sterling silver cubic zirconia womens wes...,"[9, 6, 3, 1, 10, 6, 8, 9]"
30386,30387,ayesha womens drop earrings with pastel stone ...,"[8, 8, 1, 1, 10, 4, 7]"
19849,19850,indya floral silk regular fit womens sharara -...,"[3, 8, 4, 7, 8, 1, 2]"
24910,24911,inc.5 synthetic slipon womens party wear flats...,"[7, 1, 5, 3, 6, 1, 3, 4, 9, 7]"


In [50]:
random_users = []

for i in range(10000):
    length_of_list = random.randint(0, 10)
    random_list = [random.randint(1, 10) for i in range(length_of_list)]
    random_users.append(random_list)

print(random_users)

[[3, 7, 2], [10, 3, 10, 10, 3, 9, 8, 9], [4, 9, 5, 3, 4, 2, 3, 7, 1], [8, 6, 2, 8, 7], [8, 9, 3, 2], [5, 3, 9, 9, 3, 8, 1, 3], [3, 5, 2, 2, 5, 2, 9], [], [4, 2, 9, 8, 1, 4, 7, 7], [2, 7, 4, 9, 7, 8], [2, 2, 9], [3, 9, 9, 10, 2, 7], [3, 1, 1, 9, 8, 3], [10], [5, 2, 3, 7, 4], [10], [5, 4, 1, 8, 3, 3, 6, 4], [10, 5, 2, 7, 3, 3, 5, 8], [1, 9], [3, 8, 10, 2, 2, 6, 10, 9, 7], [5], [2, 3, 9, 7, 5], [3, 6, 8, 1, 9, 10, 10], [5, 4, 7, 5, 5, 6, 7], [4, 2, 8, 4], [5, 5], [], [7, 8, 6, 4, 5, 5, 1, 2, 9], [1], [3, 1, 1, 5, 8, 7, 3], [8, 4, 1, 1, 3, 1, 7, 10, 9, 3], [3], [9, 5], [2, 9, 7, 10], [5, 4, 6, 5, 9, 1, 4, 10, 9, 9], [7, 7, 2, 1], [5, 1, 5, 7, 6, 4, 8, 10, 3], [10], [1, 7, 1, 1, 7, 2, 8, 1, 10, 6], [4, 10], [4, 6, 6, 3, 8, 2, 2, 2, 7, 5], [], [4, 5, 9, 2, 4, 5, 2], [10, 9, 4, 10, 5], [], [9, 2, 5, 2], [4, 2, 10, 4, 7, 6], [1, 6, 1, 4, 7, 2, 8, 10, 4], [4, 3, 6], [4, 4, 6, 2, 3], [6, 7, 6, 9, 5, 9], [2, 6], [5, 2, 2, 6, 7, 9], [7, 4, 8, 6, 10, 5, 7, 5, 5, 5], [2], [5, 1, 7, 9, 2, 9, 5, 4, 8,

In [52]:
print(len(random_users))

10000
